## NetworkPlot

### Set environmental variables

In order to properly load modules within this notebook from outside the repository folder, set the script **PATH** below,  e.g. ```C:/NetworkPlot```:

In [ ]:
PATH = "/path/to/NetworkPlot" # <-- optional if running from native path

In [ ]:
import importlib.util, os

if not os.path.isdir(PATH):
    PATH = os.getcwd()
PATH = os.path.realpath(PATH)

spec = importlib.util.spec_from_file_location("__init__", PATH+'/__init__.py')
init = importlib.util.module_from_spec(spec)
spec.loader.exec_module(init)

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Import functions

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py

from NetworkPlot import NetworkPlot
from nxlib import nx_readgraph, nx_readfile
from plotlib import py_plot

py.init_notebook_mode(connected=True)

### Load network data

Choose and import a network graph object to analyze below.

#### 1) Graph data file

Common graph file formats to be imported as a ```networkx.Graph()``` object. Accepts GDF/GEXF/GML/GraphML/Pajek & others.

In [ ]:
graph_file = ""

G = nx_readgraph(graph_file)

#### 2) Spreadsheet file

Reads content from file and build graph object, e.g. from `TwitterCollector` data. Accepts CSV/TAB/XLS/XLSX/TXT formats.

In [ ]:
input_file = "" # "tweets.csv"
source     = "" # "original_tweet_screen_name"
target     = "" # "retweet_screen_name"

G = nx_readfile(input_file, source=source, target=target)

#### 3) Citation Network

Reads content from folder using ```metaknowledge.RecordCollection```, split by year and assign it to a graph object.

In [ ]:
import metaknowledge as mk

In [ ]:
raw_data   = ""
start_year = 2017
end_year   = 2018

if raw_data:
    RC = mk.RecordCollection(raw_data, cached=True)
    RC = RC.yearSplit(start_year, end_year)
    G = RC.networkCitation()
    print(len(RC),'total records\n')
    for line in RC.glimpse().split('\n')[3:]:
        print(line)

### Generate network analysis

Calls NetworkPlot in order to compute centrality measures, identify modules (clusters) and render graph.

* Layouts: `circular`, `kamada_kawai`, `fruchterman_reingold`, `spring` (F-R),  `spectral`, `forceatlas2` or `random`.

* Renderers: `networkx`, `datashader`, `ds_bundled`, `plotly` or `none`.

In [ ]:
df, dfm, graphs = NetworkPlot(G,          # <-- input graph (required)
                  k=1,                    # <-- k-value for k-core filter
                  df=[],                  # <-- previously saved centrality
                  layout='forceatlas2',   # <-- from networkx or datashader
                  renderer='networkx',    # <-- library to plot graph
                  it=500,                 # <-- maximum number of iterations
                  deg=True,               # <-- degree centrality measure
                  clu=True,               # <-- clustering coefficient
                  clo=True,               # <-- closeness centrality
                  eig=True,               # <-- eigenvector centrality
                  bet=True,               # <-- betweenness centrality
                  bri=True,               # <-- bridgeness centrality
                  bro=True,               # <-- brokeness centrality
                  mod=True,               # <-- Louvain method for modularity
                  normalized=True,        # <-- all except degree centrality
                  show_labels=True,       # <-- write names above nodes
                  max_nodes=1000,         # <-- number of top nodes to render
                  max_modules=22,         # <-- number of top modules to render
                  max_labels=30,          # <-- number of top node labels to render
                  plot_modules=False,     # <-- set as True to plot modules' graphs
                  output_folder='output') # <-- optionally set the output folder name

#### Data frame from nodes

Displays data frame of centrality measures from the nodes.

In [ ]:
df

#### Data frame from modules

Displays data frame of centrality measures from identified modules.

In [ ]:
dfm

#### Correlate measures

Displays chart correlating centrality measures from the modules. **Note:** one-liner requires `plotly_express`.

In [ ]:
import plotly_express as px

In [ ]:
x = 'eigenvector'
y = 'betweenness'

px.scatter(dfm, x=x, y=y, color='cat', size='degree')#.write_html('chart.html')

#### Compress output →  `output.zip`

In [ ]:
!zip -r output.zip output

### [Download output files](output.zip)

_____
### References

* NetworkX: https://networkx.github.io

* Datashader: http://datashader.org/

* HoloViews: http://holoviews.org/

* Plotly: https://plot.ly